Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
system('curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b817262cb6114e7295fee4f73b22a3ad&view=download_cytoscape_data" -o GNPS_output_graphML.zip')
system('unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip')

specify your NAP job ID:

In [2]:
nap_id = 'c4bb6b8be9e14bdebe87c6ef3abe11f6'

load libraries

In [3]:
library('reticulate')
library('rjson')
library('igraph')


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [4]:
devtools::install_github("madeleineernst/RMolNetEnhancer", force = T)

✔  checking for file ‘/private/var/folders/fh/hzqxnlrs1gg7lmdpsfbknrqc0000gn/T/RtmpXDo2bN/remotes282858636541/madeleineernst-RMolNetEnhancer-d8a7c22/DESCRIPTION’
─  preparing ‘MolNetEnhancer’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘MolNetEnhancer_0.0.0.9000.tar.gz’
   


In [5]:
molnetenhancer <- import('pyMolNetEnhancer')
csv <- import('csv')
json <- import('json')

In [6]:
if ('clusterinfo_summary' %in% list.files('GNPS_output_graphML/') & 'DB_result' %in% list.files('GNPS_output_graphML/')){
    netfile <- paste0('GNPS_output_graphML/clusterinfo_summary/',
                        list.files('GNPS_output_graphML/clusterinfo_summary/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/DB_result/',
                        list.files('GNPS_output_graphML/DB_result/')[1])
} else if ('clusterinfosummarygroup_attributes_withIDs_withcomponentID' %in% list.files('GNPS_output_graphML/')){
    netfile <- paste0('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/',
                        list.files('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/result_specnets_DB/',
                        list.files('GNPS_output_graphML/result_specnets_DB/')[1])

} else {
    netfile <- paste0('GNPS_output_graphML/clusterinfosummary/',
                        list.files('GNPS_output_graphML/clusterinfosummary/')[1])
    gnpslibfile <- paste0('GNPS_output_graphML/result_specnets_DB/',
                        list.files('GNPS_output_graphML/result_specnets_DB/')[1])
}

In [7]:
nap <- read.csv(paste0("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task=",nap_id,
                       "&block=main&file=final_out/node_attributes_table.tsv"), sep = "\t", check.names = F)
gnpslib <- read.csv(gnpslibfile, sep='\t', check.names = F)

add all chemical structural information output as dataframe items in list

In [8]:
matches <- list(gnpslib, nap)

In [14]:
out <- molnetenhancer$unique_smiles(matches)
smiles <- py_to_r(out$df)
colnames(smiles)[1] <- 'SMILES'
smiles <- smiles[-which(smiles[,1] %in% c('N/A','n/a')),,drop = FALSE]

In [15]:
write.csv(smiles,"SMILES.csv", quote = F, row.names = F)

convert SMILES to InchiKeys

In [16]:
system('/Applications/MarvinSuite/bin/molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt')

In [17]:
ikeys <- read.csv('InchiKeys.txt',sep='\t',header = F)
colnames(ikeys)[1] = "InChIKey"
out$df <- smiles
out$df$inchikey <- ikeys[,1]
inchi_dic <- molnetenhancer$make_inchidic(out)
write.csv(ikeys,"InchiKeys.txt", quote = F, row.names = F)

retrieve ClassyFire classifcations

In [18]:
molnetenhancer$get_classifications("InchiKeys.txt")

In [19]:
py <- import_builtins()
with(py$open("all_json.json") %as% tweetfile,{
    jsondic = json$loads(tweetfile$read())
})

df <- molnetenhancer$make_classy_table(jsondic)
colnames(df)[which(colnames(df) == 'class')] <- 'CF_class'
colnames(df)[which(colnames(df) == 'smiles')] <- 'SMILES'

net <- read.csv(netfile,  sep='\t', check.names = F)
final <- molnetenhancer$molfam_classes(net,df,dict(inchi_dic))

collists <- colnames(final)[which(sapply(final, class) == 'list')]
numcols <- c('CF_NrNodes',grep('score',colnames(final),value=TRUE))

for (i in 1:length(collists)){
    final[,which(colnames(final) == collists[i])] <- unlist(final[,which(colnames(final) == collists[i])])
}
for (i in 1:length(numcols)){
    final[,which(colnames(final) == numcols[i])] <- as.numeric(as.character(final[,which(colnames(final) == numcols[i])]))
}

In [20]:
head(final)

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
0,1,343,4,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
1,3,343,4,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
2,20,343,4,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
3,1959,343,4,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
4,2,S1,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA
5,4,S2,1,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA,no matches,NA


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [21]:
write.table(final,"ClassyFireResults_Network.txt", quote = F, row.names = F,  sep = '\t')

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [22]:
length(unique(ikeys$InChIKey)[-which(unique(ikeys$InChIKey)%in% df$inchikey)])

[1] 5344

which InChIKeys could not be classified?

In [25]:
unique(ikeys$InChIKey)[-which(unique(ikeys$InChIKey)%in% df$inchikey)]

[1] InChIKey=PEWSLXRHLFWQOH-MZXGQXEMSA-N InChIKey=GYXQEZPVPQYTMA-UHFFFAOYSA-N
   [3] InChIKey=VZOUDULKCUBJHR-UHFFFAOYSA-N InChIKey=ZLVQUMWIETWGHR-LWPPXWBQSA-N
   [5] InChIKey=IAPUSMZQQSXGDI-QXHNIIHBSA-N InChIKey=BMQNSHDVIYZULR-GMKWGACXSA-N
   [7] InChIKey=ZMIRPKQRKYFMDH-UHFFFAOYSA-N InChIKey=JNRAPROKLOUIRA-UHFFFAOYSA-N
   [9] InChIKey=JUCFOMBAZKCWTF-DBUMTQGESA-N InChIKey=OFIDNKMQBYGNIW-ZKWNWVNESA-N
  [11] InChIKey=NRJAVPSFFCBXDT-UHFFFAOYSA-O InChIKey=MYKCJTMBUXHIMQ-YXLFCKQPSA-N
  [13] InChIKey=OEJGVNMSFPGDPP-BDFZTFKFSA-O InChIKey=KXYNZIYEUCUDTK-LAVZCGHOSA-N
  [15] InChIKey=MRYSHIGWBTYBNS-QEKORCQSSA-N InChIKey=TZHRSSAYNMUCJN-LHKDSFIJSA-O
  [17] InChIKey=JQFNAPFLERTHQU-ZSYXUDMDSA-N InChIKey=AZOVHLSPQFLBKM-UXUGSIJQSA-N
  [19] InChIKey=RNOUMPQRUUASAS-YWIOZPJLSA-N InChIKey=AXRXQPWGBYSWOE-PYHSYOTJSA-N
  [21] InChIKey=RSTZNFQDRKJJEI-ZOSCFRLCSA-N InChIKey=YSJPQXFWVVVLMI-AJQTZOPKSA-N
  [23] InChIKey=ALIOWXDCTBYGOF-UGDRYVHNSA-N InChIKey=RDROALZQSJZKED-JLNKQSITSA-M
  [25] InChIKey=MMYRDQLKHIDOMP-CLONMANBSA-N InChIKey=GHNFMHOEKFLHDS-JLNKQSITSA-M
  [27] InChIKey=BAWUURNLFXGLTO-GYWJBNCRSA-N InChIKey=KGPJZBJRLITRQF-GGJINPDOSA-N
  [29] InChIKey=IEIQVTKGRCQFFO-RGXQGPKTSA-N InChIKey=PWQBJCNBDUDXMV-XFCANUNOSA-N
  [31] InChIKey=LLKOVVHLWKWLBT-VCBZYWHSSA-N InChIKey=UOWRXUCVBJMAOH-BJLQDIEVSA-N
  [33] InChIKey=YPDTVIRAQMJPKU-MVIPISOYSA-O InChIKey=JDHSJCFECBVXLT-GAXRYYNESA-N
  [35] InChIKey=MZTFVLCKYQEFDJ-GOEBONIOSA-N InChIKey=IRTDDRXKLWHUSH-WZYRSQIMSA-O
  [37] InChIKey=JAMRJAUPPGMSDB-BEFSUNGCSA-N InChIKey=QKLKNRUIDIOUMG-WEYGHZABSA-P
  [39] InChIKey=QJFUYSHUUFYZLS-KWFAWWAESA-N InChIKey=DIYJLQPZULMTGY-HNBCZZSGSA-N
  [41] InChIKey=VXOZCESVZIRHCJ-LNSUSAKYSA-N InChIKey=GDOJBHVTKJOBGY-AFKKWEQQSA-N
  [43] InChIKey=HPMRFMKYPGXPEP-UHFFFAOYSA-N InChIKey=UANIKJWRSQDMPC-UHFFFAOYSA-O
  [45] InChIKey=ZJPQHHFYMMVDPZ-HEUFSPGASA-N InChIKey=YQGYDJYDIWCHOB-CYXXVJKDSA-O
  [47] InChIKey=JFBLXKBDASLQMH-HZPDHXFCSA-N InChIKey=LATVVKIIEPSSHS-UHFFFAOYSA-N
  [49] InChIKey=XAXBRBPUKYGRTP-ZDUSSCGKSA-N InChIKey=ARKNYYXVUBVCHV-FEVNIDIWSA-N
  [51] InChIKey=JBWTYVZEMQUGCH-XEOZGDDTSA-N InChIKey=KLZYXWJPRWEJSX-JASPGECPSA-N
  [53] InChIKey=GJAUAWFUYQWOPH-NCNGIYECSA-N InChIKey=IZDWXHIHOWRUTO-NTEJXHNLSA-L
  [55] InChIKey=XJSUDGJMDALOTG-GTNSWQLSSA-N InChIKey=ULUVTDOVIRYSSF-QTAQDSFMSA-N
  [57] InChIKey=QZXMUPATKGLZAP-UHFFFAOYSA-N InChIKey=BUXXJUOCOXGZNO-AXYZYVQASA-P
  [59] InChIKey=WSEQXVZVJXJVFP-UHFFFAOYSA-N InChIKey=PTGXAUBQBSGPKF-UHFFFAOYSA-N
  [61] InChIKey=TZRMVTKKQMURME-JQGMBBIASA-P InChIKey=IVHXVGOXEFRKFB-UHFFFAOYSA-N
  [63] InChIKey=BDZRJQFMMZIHRB-KQHMQFKDSA-O InChIKey=JKTCVURCUCFDGC-DDYSHWNRSA-N
  [65] InChIKey=VPRLICVDSGMIKO-SZWOQXJISA-N InChIKey=UBCVTAPCCQRPJS-UHFFFAOYSA-N
  [67] InChIKey=OJPJSOVOOAYROD-ZBFHGGJFSA-N InChIKey=DJLWHSVLAQHNTB-KTQVKEDJSA-N
  [69] InChIKey=VXOBKSYWBXNDBB-UHFFFAOYSA-P InChIKey=PLUUPKMBPOFNDA-JEFQDFFJSA-N
  [71] InChIKey=ODYLVZFQEUIVDY-KUFXCBRLSA-N InChIKey=MFDDLXWAILMNJR-UHFFFAOYSA-N
  [73] InChIKey=CLRCAFAXMVNJRH-OAHLLOKOSA-N InChIKey=GKUQGTGAOFCZFJ-UHFFFAOYSA-N
  [75] InChIKey=BVRULQCEMCYBAM-UHFFFAOYSA-N InChIKey=YWYZMPABROOXTM-BKGWKKLQSA-N
  [77] InChIKey=NHRGJVVEKNHIIE-KWJCNBFNSA-N InChIKey=ITNCNPITYLLMNG-ZFABRUEWSA-L
  [79] InChIKey=ANFSGEGICOWEAR-ZKPCAEPLSA-N InChIKey=VWYIWOYBERNXLX-KTKRTIGZSA-N
  [81] InChIKey=MOTOGXKKJSCKMT-NBGIEHNGSA-N InChIKey=WXGHOWLKEUMJEN-PJZUNXQQSA-N
  [83] InChIKey=MMWTVDJVEFZVRA-ACEXITHZSA-N InChIKey=IEZRNEGTKRQRFV-LFBNJJMOSA-N
  [85] InChIKey=FNJKRRDEBYRSCF-UDIJXASESA-N InChIKey=VWXWVRSCUGBRNL-FZBODCFUSA-O
  [87] InChIKey=OCNYQKOYWNBJFK-HUSOUSLWSA-N InChIKey=NIMPMJIXSJYVEO-UHFFFAOYSA-N
  [89] InChIKey=AOKUBKCGRXTZDV-CNTRQJEFSA-N InChIKey=RVRZDDKXZWTQSX-SSDOTTSWSA-N
  [91] InChIKey=CCMBGMOYVQDEBC-LHOUOUQOSA-N InChIKey=BZQIXQOQDMCRPD-VANHNXMTSA-N
  [93] InChIKey=IOTAOIMWBQMEAL-VEAXJEOVSA-N InChIKey=UMSAZVWWNPZQBE-VPZANPLZSA-N
  [95] InChIKey=JIPTZBYHWFNYFB-ZDUSSCGKSA-N InChIKey=ZVBKDLOCGUDHLK-HDJQEDSCSA-O
  [97] InChIKey=JMZZBVNSDNHLIV-BZKFBUQNSA-N InChIKey=IYVWUDABTOAHMQ-RNLCRUPCSA-N
  [99] InChIKey=HHINJQKIQDURLW-UTD

create graphML file

In [26]:
if (length(grep('FEATURE',list.files('GNPS_output_graphML/'))) != 0){
    graphMLfile <- paste0('GNPS_output_graphML/',list.files('GNPS_output_graphML/')[grep('FEATURE',list.files('GNPS_output_graphML/'))])
    graphML <- read_graph(graphMLfile, format = c("graphml"))
    graphML_classy <- MolNetEnhancer::make_classyfire_graphml(graphML, final)
    write_graph(graphML_classy, 'ClassyFireResults_Network.graphml', format = "graphml")
} else if (length(grep('METABOLOMICS',list.files('GNPS_output_graphML/'))) != 0) {
    graphMLfile <- paste0('GNPS_output_graphML/',list.files('GNPS_output_graphML/')[grep('METABOLOMICS',list.files('GNPS_output_graphML/'))])
    graphML <- read_graph(graphMLfile, format = c("graphml"))
    graphML_classy <- MolNetEnhancer::make_classyfire_graphml(graphML, final)
    write_graph(graphML_classy, 'ClassyFireResults_Network.graphml', format = "graphml")
} else {
    print('There is no graphML file for this GNPS molecular network job')
}